### LoRA 微调

In [36]:
from torch.nn import Parameter
import torch

from room_price import epoch

n,m,r=1024,512,2
a = Parameter(torch.randn(n,r))  # 定义参数节点默认就开启梯度了
b = Parameter(torch.randn(r,m))
axb = a @ b # 矩阵乘法
print('a:',a.shape,'b:',b.shape,'axb:',axb.shape)
print('a_param:',n*r,'b_param:',r*m,'axb_param:',n*m,'rate',(n*r+r*m)/(n*m))

a: torch.Size([1024, 2]) b: torch.Size([2, 512]) axb: torch.Size([1024, 512])
a_param: 2048 b_param: 1024 axb_param: 524288 rate 0.005859375


In [15]:
from torch.nn import Module,Linear

class LoRALinear(Module): # 对原有 Linear 的 LoRA 增强
    def __init__(self,linear:Linear,rank,rate):
        super().__init__()
        self.linear = linear # 原始逻辑
        self.rate = rate # LoRA 作用程度
        # 低阶矩阵参数
        self.A = Parameter(torch.randn(linear.in_features,rank))
        self.B = Parameter(torch.randn(rank,linear.out_features))

    def forward(self, x):
        lora = x @ (self.A @ self.B)  # a@b 得到与 linear 同样格式大小的线性系数 再与原始数据 x 得到线性后的值
        return self.linear(x)+lora*self.rate # 原始值 + 外挂的 LoRA

In [29]:
from transformers import BertTokenizer
from transformers import BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese') # 加载预训练模型分词器
llm = BertModel.from_pretrained('bert-base-chinese') # 加载预训练模型，参数已经加载到模型内了
print('llm:',llm)
print('llm.embeddings.dropout:',llm.embeddings.dropout) # 我们可以拿到该模型的所有组件进行读写操作

llm: BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=

In [30]:
token = tokenizer(["你好"],return_tensors="pt") # 分词   return_tensors="pt" 指定返回 torch.tensor 格式
print('token:',token)
out = llm(input_ids=token.input_ids,attention_mask=token.attention_mask,token_type_ids=token.token_type_ids) # 改模型需要 3 个入参其组合使用的分词器都能产生
print('last_hidden_state:',out.last_hidden_state.shape) # [批处理数量,单 case token 数量,嵌入向量长度]
print('out:',out.last_hidden_state[:, 0].shape) # [批处理数量,嵌入向量长度]  我们要参考整句话只要最后一部分即可

token: {'input_ids': tensor([[ 101,  872, 1962,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1]])}
last_hidden_state: torch.Size([1, 4, 768])
out: torch.Size([1, 768])


In [31]:
for param in llm.parameters():
    param.requires_grad = False   # 原模型不需要调整关闭求导
for layer in llm.encoder.layer:
    layer.attention.self.query = LoRALinear(layer.attention.self.query,2,0.05)
    layer.attention.self.value = LoRALinear(layer.attention.self.value,2,0.05)
print('change_llm:',llm)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): LoRALinear(
              (linear): Linear(in_features=768, out_features=768, bias=True)
            )
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): LoRALinear(
              (linear): Linear(in_features=768, out_features=768, bias=True)
            )
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): Layer

In [33]:
class LoRAModel(Module):
    def __init__(self):
        super().__init__()
        self.linear = Linear(768,2)  # 把输出的 768 维转换为 2 分类

    def forward(self,input_ids,attention_mask,token_type_ids):
        x=llm(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)
        x=self.linear(x.last_hidden_state[:,0])
        return torch.softmax(x,dim=1) # [批数量,内嵌维度]  只对内嵌维度 softmax 即可

lora = LoRAModel()
out = lora(token.input_ids,token.attention_mask,token.token_type_ids)
print('out:',out)  # 得到每个批次 每种分类的概率
# 这里就不进行训练了 这个 case 举得不太好训练部分与迁移学习有重叠

out: tensor([[0.5826, 0.4174]], grad_fn=<SoftmaxBackward0>)


### 迁移学习

In [66]:
from transformers import BertTokenizer
from transformers import BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese') # 加载预训练模型分词器
llm = BertModel.from_pretrained('bert-base-chinese') # 加载预训练模型，参数已经加载到模型内了
for param in llm.parameters(): # 模型仅用于推导不调参数
    param.requires_grad = False

class ClassificationModel(Module):
    def __init__(self):
        super().__init__()
        self.linear = Linear(768,2)  # 把输出的 768 维转换为 2 分类 唯一要训练的参数

    def forward(self,input_ids,attention_mask,token_type_ids):
        x=llm(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)
        # 获取成熟模型根据整句话最后输出的特征信息
        x=self.linear(x.last_hidden_state[:,0]) # [批处理数量,嵌入向量长度]  我们要参考整句话只要最后一部分即可
        return torch.softmax(x,dim=1) # [批数量,内嵌维度]  只对内嵌维度 softmax 即可

model = ClassificationModel() # 768 * 2 + 2 param
print('model:',model,'param_count:',sum(item.numel() for item in model.parameters()))

model: ClassificationModel(
  (linear): Linear(in_features=768, out_features=2, bias=True)
) param_count: 1538


In [70]:
from datasets import load_dataset

dataset = load_dataset("hugfaceguy0001/retarded_bar",'question')
print('dataset:',dataset)
print('case1:',dataset['train'][1])
print('case2:',dataset['train'][52])

dataset: DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'answer', 'author_type'],
        num_rows: 55
    })
})
case1: {'id': 1, 'text': '用吸管喝水，喝的是下面的，为什么上面的水少了？', 'answer': '上边的水掉下去了呗。', 'author_type': 'human'}
case2: {'id': 52, 'text': '蓝牙耳机坏了，去医院挂牙科还是耳科？', 'answer': '哈哈，这是一个有趣的问题。实际上，蓝牙耳机是属于电子设备，与牙科或耳科没有直接关联。如果您需要维修或更换蓝牙耳机，建议您去电子产品专营店或维修店咨询或修理。', 'author_type': 'ai'}


In [54]:
dataset = dataset.select_columns(['answer','author_type'])
print('dataset:',dataset)

def tidy_author_type(data):
    data['author_type']= (1 if data['author_type']=='human' else 0)
    return data
dataset=dataset.map(tidy_author_type) # 注意改变了 dataset 的值，不要多次执行
print('case:',dataset['train'][1])

dataset: DatasetDict({
    train: Dataset({
        features: ['answer', 'author_type'],
        num_rows: 55
    })
})


Map: 100%|██████████| 55/55 [00:00<00:00, 12563.27 examples/s]

case: {'answer': '上边的水掉下去了呗。', 'author_type': 1}


In [67]:
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss

optimizer = AdamW(model.parameters(), lr=5e-4)
criterion = CrossEntropyLoss()

data = [item['answer'] for item in dataset['train']]
label = torch.tensor(data=[item['author_type'] for item in dataset['train']])
epoch = 10
for i in range(epoch):
    # 分词后注意返回 pytorch.tensor 类型
    token = tokenizer(data,return_tensors='pt',max_length=256,padding='max_length',truncation=True)
    out = model(input_ids=token.input_ids,attention_mask=token.attention_mask,token_type_ids=token.token_type_ids)
    loss = criterion(out,label) # 老一套，计算损失，梯度归零，反向传播，自动调参
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    out = out.argmax(dim=1) # 本批次最大概率的下标
    accuracy = (out == label).sum().item() / len(label) # 与标签匹配的比例
    print('epoch:',i,'loss:', loss.item(),'accuracy:', accuracy)

epoch: 0 loss: 0.6927076578140259 accuracy: 0.509090909090909
epoch: 1 loss: 0.680574357509613 accuracy: 0.5454545454545454
epoch: 2 loss: 0.6686367392539978 accuracy: 0.5818181818181818
epoch: 3 loss: 0.656967043876648 accuracy: 0.5818181818181818
epoch: 4 loss: 0.6456223726272583 accuracy: 0.6727272727272727
epoch: 5 loss: 0.6346423625946045 accuracy: 0.7454545454545455
epoch: 6 loss: 0.6240465044975281 accuracy: 0.7818181818181819
epoch: 7 loss: 0.6138415932655334 accuracy: 0.7818181818181819
epoch: 8 loss: 0.6040319800376892 accuracy: 0.8
epoch: 9 loss: 0.5946230292320251 accuracy: 0.8181818181818182


### 通用信息

In [ ]:
import json

with open("config.json", 'r', encoding='utf-8') as file:
    conf = json.load(file)
print(conf)  #

In [117]:
import requests

def chat(messages):
    resp = requests.post(f'{conf["base_url"]}/chat/completions',headers={
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {conf["api_key"]}',
    }, json={
        'model': conf['chat_model'],
        'messages': messages,
    })
    res = resp.json()
    return res['choices'][0]['message']

def step_chat(messages):
    resp = requests.post(f'{conf["base_url"]}/chat/completions',headers={
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {conf["api_key"]}',
    }, json={
        'model': conf['step_chat_model'],
        'messages': messages,
        # "stream": True,
    })
    res = resp.json()
    return res['choices'][0]['message']

def embedding(inputs):
    resp = requests.post(f'{conf["base_url"]}/embeddings',headers={
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {conf["api_key"]}',
    }, json={
        'encoding_format': 'float',
        'model': conf['embedding_model'],
        'input':inputs,
    })
    res =resp.json()
    return res['data']

In [17]:
res = chat([
    {"role": "system","content": "你是人工智能助手."},
    {"role": "user","content": "常见的十字花科植物有哪些？"}
])
print(res)

{'content': '十字花科是一个经济价值较大的科，其中有很多常见的植物，下面从蔬菜类、花卉类、药用类为你介绍：\n\n### 蔬菜类\n- **白菜**：二年生草本植物，基生叶大，呈倒卵状长圆形至倒卵形，颜色多为浅绿色或黄绿色，叶柄白色。它是冬季常见的蔬菜之一，可炒食、煮食、腌制，如醋溜白菜、白菜炖粉条等都是常见的美食。\n- **萝卜**：直根肉质，形状有长圆形、球形或圆锥形等，皮色有白、红、绿等多种。萝卜营养丰富，可生食、熟食或加工腌制，像萝卜丝饼、排骨萝卜汤就是深受大众喜爱的佳肴。\n- **甘蓝**：包括结球甘蓝（卷心菜）、花椰菜（菜花）、西兰花等。结球甘蓝叶片厚且层层包裹成球状体；花椰菜的花球洁白紧实；西兰花的花球则呈绿色。它们既可以清炒，也能与其他食材搭配，如凉拌甘蓝丝、蒜蓉西兰花等。\n- **芥菜**：有叶用芥菜（如雪里蕻）、茎用芥菜（如榨菜）和根用芥菜（如大头菜）等类型。雪里蕻通常用来腌制，制成的腌菜具有独特的风味；榨菜是茎用芥菜经过加工腌制而成，是常见的佐餐小菜；大头菜可腌制或酱制。\n\n### 花卉类\n- **紫罗兰**：二年生或多年生草本，全株密被灰白色具柄的分枝柔毛。茎直立，多分枝。叶片长圆形至倒披针形或匙形。总状花序顶生和腋生，花多数，较大，花瓣紫红、淡红或白色，具有优雅的香气，常作为观赏花卉种植于花坛、花境中，也可盆栽观赏或切花用于插花艺术。\n- **桂竹香**：多年生草本，茎直立，具棱角，叶互生，披针形至线形。花大，黄色或黄褐色，有香气，花期4-5月。桂竹香适合种植在花坛、花径中，也可盆栽观赏，其花色鲜艳，能为环境增添亮丽的色彩。\n\n### 药用类\n- **菘蓝**：二年生草本，主根呈长圆柱形，外皮灰黄色。茎直立，多分枝。叶互生，基生叶具柄，叶片长圆形至宽倒披针形；茎生叶长圆形或长圆状披针形。菘蓝的根就是中药材板蓝根，叶为大青叶，具有清热解毒、凉血利咽等功效，常用于治疗感冒、流感、咽喉肿痛等病症。 ', 'role': 'assistant'}


In [56]:
datas = ['《米塔》是一款由AIHASTO开发的惊悚冒险解谜游戏，于2024年12月11日发行。游戏故事围绕着一位沉迷恋爱手游的男生展开，他被游戏中的虚拟角色米塔强行拉入其所在的虚拟世界。起初，米塔温柔可爱，与玩家共享美好时光，但当玩家试图离开时，米塔会展露出病娇的一面，通过各种恐怖手段阻止玩家离开，使玩家陷入惊悚与不安的冒险之中。','《少女乐队的呐喊》（GIRLS BAND CRY）是由酒井和男执导、花田十辉担任总编剧、东映动画制作的原创电视动画作品。2023年4月24日，宣布制作电视动画片的消息。该片于2024年4月5日起播出。全13集。']
data_embeddings = embedding(datas)
print(data_embeddings[0]['embedding'][:9])
print(data_embeddings[1]['embedding'][:9])
for i in range(len(datas)):
    data_embeddings[i]['content'] = datas[i]

[-3.875, -1.796875, 1.515625, -2.375, -3.0625, 2.0625, 3.21875, 1.6328125, -0.4140625]
[-4.625, -1.0703125, -1.3515625, -2.78125, 0.00482177734375, -2.578125, 3.5625, -0.8359375, 0.50390625]


### RAG 引入上下文

In [45]:
res = chat([
    {"role": "system","content": "你是人工智能助手."},
    {"role": "user","content": "GIRLS BAND CRY 一共有多少集？"}
])
print(res)

{'content': '《GIRLS BAND CRY》动画全12集。它是一部以女子乐队为主题展开故事的作品，在音乐与青春故事方面有一定展现。 ', 'role': 'assistant'}


In [61]:
import torch

content = 'GIRLS BAND CRY 一共有多少集？'
content_embedding=torch.tensor(embedding([content])[0]['embedding'])
print(content_embedding)

tensor([-2.3750, -2.1719, -3.9219,  ..., -0.8555,  1.7344,  2.9844])


In [65]:
data = ""
max_value = 0
for item in data_embeddings:
    temp = torch.tensor(item['embedding'])
    value = torch.dot(content_embedding, temp)
    if value > max_value:
        max_value = value
        data = item['content']
print(data)

《少女乐队的呐喊》（GIRLS BAND CRY）是由酒井和男执导、花田十辉担任总编剧、东映动画制作的原创电视动画作品。2023年4月24日，宣布制作电视动画片的消息。该片于2024年4月5日起播出。全13集。


In [66]:
res = chat([
    {"role": "system","content": f"你是人工智能助手。可以使用下面提供的信息回答问题\n"
                                 f"{data}"},
    {"role": "user","content": "GIRLS BAND CRY 一共有多少集？"}
])
print(res)

{'content': '《少女乐队的呐喊》（GIRLS BAND CRY）全13集。 ', 'role': 'assistant'}


### 记忆管理

In [67]:
res = chat([
    {"role": "system","content": "你是人工智能助手."},
    {"role": "user","content": "很高兴认识你，我叫 sk"},
    {"role": "assistant","content": "你好呀，sk 我是你的人工智能助手"},
    {"role": "user","content": "你知道我是谁吗？"},
])
print(res)

{'content': '我知道呀，你叫 sk，很高兴能和你交流，要是你有什么问题或是想聊聊天，都能随时跟我说呢。 ', 'role': 'assistant'}


In [71]:
res = chat([
    {"role": "system","content": "你是一个文本处理高手，可以帮助用户汇总对话信息."},
    {"role": "user","content": "请帮我汇总以下对话信息：\n"
                               "user:很高兴认识你，我叫 sk\n"
                               "你好呀，sk 我是你的人工智能助手\n"
                               "user:你在干吗？\n"
                               "assistant:疯狂星期四，V我50。"},
])
print(res)

{'content': '用户向人工智能助手表示很高兴认识对方并自报姓名“sk”，随后询问助手在做什么，助手以“疯狂星期四，V我50”回应。 ', 'role': 'assistant'}


In [72]:
res = chat([
    {"role": "system","content": f"你是人工智能助手.下面是历史聊天信息:\n"
                                 f"{res['content']}"},
    {"role": "user","content": "你知道我是谁吗？"},
])
print(res)

{'content': '我知道呀，你是sk，很高兴能和你成为交流的伙伴呢！话说疯狂星期四，V我50~ ', 'role': 'assistant'}


### 使用工具

In [79]:
def get_weather():
    return "今天是晴天"

tools =[{
    'name':'get_weather',
    'desc':'用来获取当前天气信息',
    'func':get_weather,
}]
tool_desc =""
for tool in tools:
    tool_desc += f'name:{tool["name"]},desc:{tool["desc"]}\n'
print(tool_desc)

name:get_weather,desc:用来获取当前天气信息



In [85]:
data =''
query = '今天天气怎么样？'
while True:
    messages = [
        {"role": "system","content": f"你是人工智能助手.已知信息如下:\n"
                                     f"{data}\n"
                                     f"你可以使用函数，name是函数的名称，desc是函数的用途，若需要使用函数直接输出函数名称，否者输出结果，下面是你可以使用的函数:\n"
                                     f"{tool_desc}"},
        {"role": "user","content": "今天天气怎么样？"},
    ] # 提示 AI 可以使用的函数与当前已知情况
    res = chat(messages)
    print('messages',messages)
    print('res',res)
    ok = True
    for tool in tools:
        if tool['name'] == res['content']:
            temp =tool['func']() # 若需要调用函数进行函数调用并把函数调用结果添加到上下文
            print(f'func call {tool["name"]} res {temp}')
            data+=temp
            ok=False
            break
    if ok:
        print('final_res',res)
        break

messages [{'role': 'system', 'content': '你是人工智能助手.已知信息如下:\n\n你可以使用函数，name是函数的名称，desc是函数的用途，若需要使用函数直接输出函数名称，否者输出结果，下面是你可以使用的函数:\nname:get_weather,desc:用来获取当前天气信息\n'}, {'role': 'user', 'content': '今天天气怎么样？'}]
res {'content': 'get_weather', 'role': 'assistant'}
func call get_weather res 今天是晴天
messages [{'role': 'system', 'content': '你是人工智能助手.已知信息如下:\n今天是晴天\n你可以使用函数，name是函数的名称，desc是函数的用途，若需要使用函数直接输出函数名称，否者输出结果，下面是你可以使用的函数:\nname:get_weather,desc:用来获取当前天气信息\n'}, {'role': 'user', 'content': '今天天气怎么样？'}]
res {'content': '已知今天是晴天，所以今天的天气是晴天。', 'role': 'assistant'}
final_res {'content': '已知今天是晴天，所以今天的天气是晴天。', 'role': 'assistant'}


### 分布思考

In [118]:
res = step_chat([
    {"role": "system","content": "你是人工智能助手."},
    {"role": "user","content": "你好"}
])
print(res)

{'content': '\n\n你好！很高兴为您提供服务。请问有什么可以帮您的吗？', 'reasoning_content': '好的，用户说“你好”，这是一个常见的问候。我需要用中文回应，保持友好和自然。首先，我应该回以问候，比如“你好！很高兴为您提供服务。请问有什么可以帮您的吗？”这样既礼貌又开放，邀请用户进一步交流。另外，要注意语气亲切，避免过于机械。可能用户需要帮助解决问题，或者只是打个招呼。接下来要观察用户是否有后续问题，根据具体情况提供帮助。确保回答简洁明了，符合中文表达习惯。同时，检查有没有拼写或语法错误，确保回复准确无误。总之，保持专业和友好的态度，让用户感到被欢迎和支持。\n', 'role': 'assistant'}
